In [1]:
import requests
from bs4 import BeautifulSoup

# URL of the page you want to scrape
url = 'https://unrae.it/dati-statistici/immatricolazioni?page=1'

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Now, you need to find and extract the specific data you're interested in.
    # This involves identifying the HTML elements (tags) that contain the data.
    # For demonstration, let's say we want to extract data within <div> tags with a specific class name.
    # You'll need to replace 'div' and 'class_name' with actual tag names and class names found in the webpage.
    data_elements = soup.find_all('div', class_='class_name')
    
    for element in data_elements:
        # Extract and print the text content of each element
        # You might need to adjust this based on how the data is structured within the element
        print(element.text.strip())
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


In [3]:
import requests
from lxml import html

# URL of the page you want to scrape
url = 'https://unrae.it/dati-statistici/immatricolazioni?page=1'

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    tree = html.fromstring(response.content)
    
    # Use the provided XPath to extract the desired element(s)
    # Adjust the XPath if necessary based on the actual data you need
    xpath = '/html/body/div[2]/div/div[1]/div[4]'
    elements = tree.xpath(xpath)
    
    for element in elements:
        # Extract and print the text content of each element
        # You might need to adjust this based on the structure within the element
        print(element.text_content().strip())
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

Immatricolazioni Veicoli Industriali - Febbraio 2024
                                                L'UNRAE comunica la stima delle immatricolazioni di veicoli industriali (con massa complessiva maggiore di 3,5t)
                                                Pubblicato in data: 11 marzo 2024
                                                
                                                    Periodo di riferimento dei dati:
                                                                                                            febbraio 2024 
                                                                                                                                                
                                            
                                                                                    
                                                Immatricolazioni di autovetture per provincia di residenza del proprietario – Febbraio 2024
                                

In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os

def download_pdf(link, directory='pdfs'):
    response = requests.get(link)
    filename = link.split('/')[-1]
    filepath = os.path.join(directory, filename)
    with open(filepath, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {filename} to {directory}/")

def find_and_download_pdfs(url, container_css_selector):
    # Ensure the download directory exists
    os.makedirs('pdfs', exist_ok=True)

    # Fetch the page
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        container = soup.select_one(container_css_selector)
        if container:
            links = [a['href'] for a in container.find_all('a', href=True)]
            for link in links:
                full_link = urljoin(url, link)
                if full_link.lower().endswith('.pdf'):
                    download_pdf(full_link)
                else:
                    print(f"Non-PDF link found, needs manual processing: {full_link}")
        else:
            print("Container not found on the page.")
    else:
        print(f"Failed to retrieve page. Status code: {response.status_code}")

base_url = 'https://unrae.it/dati-statistici/immatricolazioni?page='
container_css_selector = 'div.container > div:nth-of-type(4)'

# Loop through all 149 pages
for page in range(1, 150):
    current_url = f"{base_url}{page}"
    print(f"Processing {current_url}")
    find_and_download_pdfs(current_url, container_css_selector)


Processing https://unrae.it/dati-statistici/immatricolazioni?page=1
Non-PDF link found, needs manual processing: https://unrae.it/pubblicazioni/pocket-mercato
Non-PDF link found, needs manual processing: https://unrae.it/pubblicazioni/pocket-emissioni
Non-PDF link found, needs manual processing: https://unrae.it/pubblicazioni/book-statistiche-annuali
Non-PDF link found, needs manual processing: https://unrae.it/pubblicazioni/sintesi-statistica
Processing https://unrae.it/dati-statistici/immatricolazioni?page=2
Non-PDF link found, needs manual processing: https://unrae.it/pubblicazioni/pocket-mercato
Non-PDF link found, needs manual processing: https://unrae.it/pubblicazioni/pocket-emissioni
Non-PDF link found, needs manual processing: https://unrae.it/pubblicazioni/book-statistiche-annuali
Non-PDF link found, needs manual processing: https://unrae.it/pubblicazioni/sintesi-statistica
Processing https://unrae.it/dati-statistici/immatricolazioni?page=3
Non-PDF link found, needs manual pro

In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os

def download_pdf(link, directory='pdfs'):
    response = requests.get(link, stream=True)
    filename = link.split('/')[-1]
    filepath = os.path.join(directory, filename)
    with open(filepath, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f"Downloaded {filename} to {directory}/")

def find_pdf_link_and_download(article_url, pdf_xpath):
    response = requests.get(article_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Convert the XPath to a CSS selector for PDF link, this is a placeholder, needs manual conversion
        pdf_link_selector = "div.content > p:nth-of-type(2) > b > a" # This is a guessed selector, adjust based on actual page structure
        pdf_link_element = soup.select_one(pdf_link_selector)
        if pdf_link_element and 'href' in pdf_link_element.attrs:
            pdf_url = urljoin(article_url, pdf_link_element['href'])
            download_pdf(pdf_url)
        else:
            print("PDF link not found in the article.")
    else:
        print(f"Failed to retrieve article page. Status code: {response.status_code}")

def process_main_page(main_page_url, container_css_selector):
    # Fetch the main page
    response = requests.get(main_page_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        container = soup.select_one(container_css_selector)
        if container:
            article_links = [a['href'] for a in container.find_all('a', href=True)]
            for article_link in article_links:
                full_article_url = urljoin(main_page_url, article_link)
                find_pdf_link_and_download(full_article_url, pdf_xpath='/html/body/div[2]/div/div[1]/p[2]/b/a') # Placeholder XPath
        else:
            print("Container not found on the main page.")
    else:
        print(f"Failed to retrieve main page. Status code: {response.status_code}")

base_url = 'https://unrae.it/dati-statistici/immatricolazioni?page='
container_css_selector = 'div.container > div:nth-of-type(4)' # Adjusted based on your XPath for the container

# Assuming there are 149 pages to process
for page in range(1, 150):
    current_main_page_url = f"{base_url}{page}"
    print(f"Processing main page: {current_main_page_url}")
    process_main_page(current_main_page_url, container_css_selector)


Processing main page: https://unrae.it/dati-statistici/immatricolazioni?page=1
PDF link not found in the article.
PDF link not found in the article.
PDF link not found in the article.
PDF link not found in the article.
Processing main page: https://unrae.it/dati-statistici/immatricolazioni?page=2
PDF link not found in the article.
PDF link not found in the article.
PDF link not found in the article.
PDF link not found in the article.
Processing main page: https://unrae.it/dati-statistici/immatricolazioni?page=3
PDF link not found in the article.
PDF link not found in the article.
PDF link not found in the article.
PDF link not found in the article.
Processing main page: https://unrae.it/dati-statistici/immatricolazioni?page=4
PDF link not found in the article.
PDF link not found in the article.
PDF link not found in the article.
PDF link not found in the article.
Processing main page: https://unrae.it/dati-statistici/immatricolazioni?page=5
PDF link not found in the article.
PDF link n

In [6]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os

def download_pdf(link, directory='pdfs'):
    response = requests.get(link, stream=True)
    filename = os.path.basename(link)
    filepath = os.path.join(directory, filename)
    with open(filepath, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Downloaded {filename} to {directory}/")

def get_article_links(main_page_url, container_selector):
    response = requests.get(main_page_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        container = soup.select_one(container_selector)
        if container:
            return [a['href'] for a in container.select('div > a')]
        else:
            print("Article container not found.")
            return []
    else:
        print(f"Failed to retrieve main page. Status code: {response.status_code}")
        return []

def find_and_download_pdf(article_url, pdf_selector):
    response = requests.get(article_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        pdf_link_element = soup.select_one(pdf_selector)
        if pdf_link_element and 'href' in pdf_link_element.attrs:
            pdf_url = urljoin(article_url, pdf_link_element['href'])
            download_pdf(pdf_url)
        else:
            print(f"PDF link not found in article: {article_url}")
    else:
        print(f"Failed to retrieve article page. Status code: {response.status_code}")

def process_main_pages(base_url, pages, container_selector, pdf_selector):
    for page in range(1, pages + 1):
        main_page_url = f"{base_url}?page={page}"
        print(f"Processing page: {page}")
        article_links = get_article_links(main_page_url, container_selector)
        for article_link in article_links:
            full_article_url = urljoin(base_url, article_link)
            find_and_download_pdf(full_article_url, pdf_selector)

base_url = 'https://unrae.it/dati-statistici/immatricolazioni'
pages = 149
container_selector = 'body > div.container > div > div.col-xs-12.col-md-8.unrae_cat_box > div.cat_art_container'
pdf_selector = 'body > div.container > div > div.col-xs-12.col-md-8.unrae_art_box > p:nth-child(5) > b > a'

process_main_pages(base_url, pages, container_selector, pdf_selector)


Processing page: 1
Downloaded Veicoli Industriali febbraio 2024_65eec7387c0df.pdf to pdfs/
Downloaded 13 MIT Provincia_Febbraio 2024_65e1edfcc9d3b.pdf to pdfs/
Downloaded 07 Top 10 per alimentazione_Febbraio 2024_65e1ee0cbab9c.pdf to pdfs/
Downloaded 08 Imm BEV modello Febbraio 2024_65e1ee27ed42e.pdf to pdfs/
Downloaded 10 Top Ten per segmento_Febbraio_2024_65e6dd2887ffa.pdf to pdfs/
Downloaded 05 Imm fasce CO2 canali Febbraio_2024_65e1ee41a8a09.pdf to pdfs/
Downloaded 04 Struttura del Mercato_Febbraio 2024_65e6dce161a3e.pdf to pdfs/
Downloaded 06 Top 50_Febbraio_2024_65e1ee986d93b.pdf to pdfs/
Downloaded 03 Gruppi Febbraio 2024_65e20c97640fa.pdf to pdfs/
Downloaded 02 Marca Febbraio 2024_65e2016ea99b8.pdf to pdfs/
Processing page: 2
Downloaded mercatoeu_65d47cf7c8308.pdf to pdfs/
Downloaded Stima autocarri e struttura_65cb1eea80ea5.pdf to pdfs/
Downloaded Veicoli Industriali gennaio 2024_65c5e447afcfe.pdf to pdfs/
Downloaded 13 MIT Provincia_Gennaio 2024_65bb9a8b5609f.pdf to pdfs/
Dow